### Task 1: A classification example: fetal heart condition diagnosis
The UCI Machine Learning Repository contains several datasets that can be used to investigate different machine learning algorithms. In this exercise, we'll use a dataset of fetal heart diagnosis. The dataset contains measurements from about 2,600 fetuses. This is a classification task, where our task is to predict a diagnosis type following the FIGO Intrapartum Fetal Monitoring Guidelines: normal, suspicious, or pathological.


In [1]:
## Importing Libs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier , GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold , cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score ,mean_squared_error
from sklearn.preprocessing import LabelEncoder
import os

In [2]:
Vilja = pd.read_csv('Vilja.csv', skiprows=0)
Vilja

,degenerate_application_id,ssn,kids_number,housing_cost
0,5004437,6243049928,2.0,7883.0
1,5004890,8869284425,2.0,3100.0
2,5004973,6974071364,1.0,8200.0
3,5005103,7252112375,2.0,3200.0
4,5005152,8066081509,0.0,3000.0
...,...,...,...,...
1271848,8803951,8255499,0.0,3000.0
1271849,8803953,8811115917,2.0,1500.0
1271850,8803991,6209281010,0.0,NaN
1271851,8804158,12283347,0.0,9119.0


I need python version 3.9.17 in order to run 
import pymssql

In [3]:
import pymssql  # need python version 3.9.16 to run this
import pandas as pd

server = 'reporting-db.nystartfinans.net'
database = 'reporting-db'
username = 'Andreas'
password = 'nCq8Sg@1lYnd(E'

conn = pymssql.connect(server, username, password, database)



# Read the SQL file from the specified directory
file_path = "/Users/andreasnilsson/Desktop/PD 2/DATA/CO Min score join - APL CBR MLP.sql"

with open(file_path, 'r') as f:
    sql = f.read()

# Execute the query and load data into a DataFrame

co = pd.read_sql(sql, conn)


# Read the SQL file from the specified directory
file_path = "/Users/andreasnilsson/Desktop/PD 2/DATA/MA Correct join - APL CRB-MLP.sql"

with open(file_path, 'r') as f:
    sql = f.read()

# Execute the query and load data into a DataFrame
main = pd.read_sql(sql, conn)



# Close the connection
conn.close()




/var/folders/xv/px633rpn27q5n8m3579ly8dm0000gn/T/ipykernel_68850/3919157578.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  co = pd.read_sql(sql, conn)
/var/folders/xv/px633rpn27q5n8m3579ly8dm0000gn/T/ipykernel_68850/3919157578.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  main = pd.read_sql(sql, conn)


In [4]:


def drop_columns(df: pd.DataFrame, columns_to_drop: list) -> pd.DataFrame:
    """
    Drop specified columns from a DataFrame.

    Parameters:
    - df: Input DataFrame.
    - columns_to_drop: List of column names to drop.

    Returns:
    - DataFrame with specified columns dropped.
    """
    return df.drop(columns=columns_to_drop, errors='ignore')



droplist = ['RowNumber','RowNum']                                                       ## SAme as Application ID

main = drop_columns(main, droplist)
co = drop_columns(co, droplist)
Vilja = drop_columns(Vilja, ['ssn'])




In [5]:
## filter away if we dont have UC score on some , now its approximatly a bit more than 1 %
#main[pd.isna(main.UCScore)]
main = main[~pd.isna(main.UCScore)]

main

,Ever90,AccountNumber,ApplicationID,SSN_A,IsMainApplicant,HasCoapp,ReceivedDate,DisbursedDate,Amount,StartupFee,...,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards
227,0,5004296,5004296,69C8A21CAFFDEE2706ECB2A6747E9A99BB928C9CB1E18A...,1,0,2017-01-24,2017-01-30,150000.0,995.0,...,None,None,None,2,0,23.56,Nej,None,None,0
234,0,5004551,5004551,E63A283916669FA666387284A43561E1DD68CE044FE9D9...,1,0,2017-01-26,2017-02-24,75000.0,495.0,...,0,None,None,3,0,72.15,Nej,None,None,0
235,0,5004569,5004569,2554C473AF4A2215342B0EC264551936C88479DF897459...,1,0,2017-01-26,2017-02-22,170000.0,1495.0,...,0,None,None,2,0,96.64,Nej,None,None,0
242,0,5005251,5005251,87412813A6D41199A6659090605F6C3BDC27F05F561DB4...,1,0,2017-02-01,2017-02-20,210000.0,995.0,...,0,None,None,1,0,98.33,Nej,None,None,0
245,0,5005780,5005780,017496BA0F0CA9EAFF8DC9B2B78FFBF893EDDE7A48DC0E...,1,0,2017-02-07,2017-02-14,200000.0,995.0,...,0,None,None,14,33234,79.98,Ja,None,None,30309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7628,0,7828411,8441278,0792BA7E8AF4D0178EC5342D2E2E046C5BD4189A319568...,1,0,2022-10-27,2022-10-28,285000.0,1995.0,...,0,1,0,5,0,91.83,Nej,None,None,0
7629,0,7828445,8441645,294FC9BDBD370CF3BC687DD244CAF84738895D52FD60CD...,1,0,2022-10-27,2022-10-28,130000.0,995.0,...,0,None,None,1,0,77.07,Nej,None,None,0
7630,0,7828767,8442729,C73B15E89D0AB74790DB9F5DE1BCB5C5E03940D973032D...,1,0,2022-10-28,2022-10-31,60000.0,995.0,...,0,None,None,4,0,98.89,Ja,None,None,0
7631,0,7829377,8444625,A819F8D4148ACF432FC4E8E8D8A71EFC23D94C89676DB0...,1,0,2022-10-30,2022-10-31,130000.0,295.0,...,0,1,0,1,0,35.00,Nej,None,None,0


In [6]:
co

co[pd.isna(co.UCScore)]

,Ever90,AccountNumber,ApplicationID,SSN_A,IsMainApplicant,HasCoapp,ReceivedDate,DisbursedDate,Amount,PaymentRemarks,...,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards


In [7]:
len((set(co.AccountNumber)))

3479

1. Are dupplicated ApplicationID in each df?

In [8]:


mask = co['AccountNumber'].isin(main['AccountNumber'])
print(set(mask))


{False}


2. All accounts are distinct in each df

In [9]:
main[main['AccountNumber'].duplicated(keep=False)]

,Ever90,AccountNumber,ApplicationID,SSN_A,IsMainApplicant,HasCoapp,ReceivedDate,DisbursedDate,Amount,StartupFee,...,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards


In [10]:
main[main.AccountNumber ==  '7798325'].Inquiries12M



7246    5
Name: Inquiries12M, dtype: object

main

In [11]:
# Merging on 'AccountNumber' to find common values
common_accounts = pd.merge(main, co, on='AccountNumber', how='inner')
common_accounts

,Ever90_x,AccountNumber,ApplicationID_x,SSN_A_x,IsMainApplicant_x,HasCoapp_x,ReceivedDate_x,DisbursedDate_x,Amount_x,StartupFee,...,SharedVolumeUnsecuredLoans_y,NumberOfInstallmentLoans_y,SharedVolumeInstallmentLoans_y,NumberOfCreditAccounts_y,SharedVolumeCrerditAccounts_y,UtilizationRatio_y,CreditAccountOverdraft_y,NumberOfMortgageLoans_y,SharedVolumeMortgageLoans_y,SharedVolumeCreditCards_y


In [12]:
main['Applicationtype'] = 0

co['Applicationtype'] = np.where(
    (co['HasCoapp'] == 1) & (co['CoappSameAddress'] == 1), 1,
    np.where(
        (co['HasCoapp'] == 1) & (co['CoappSameAddress'] == 0), 2,
        np.nan  # This is for the condition when neither of the above conditions is True. You can replace `np.nan` with any default value if needed.
    )
)


df = pd.concat([main ,co])




In [13]:
print(len(main))
print(len(co))

print(len(main) +len(co) ) 

print(len(df))


7204
3479
10683
10683


In [14]:
df.CivilStatus.value_counts()

CivilStatus
1    6027
4    1619
2    1405
7    1171
9     166
5      47
P       1
Name: count, dtype: int64

In [15]:
df.MaritalStatus.value_counts()


MaritalStatus
SINGLE      4195
PARTNER     2588
MARRIED     1972
None         985
             373
DIVORCED     265
WIDOWED       58
Name: count, dtype: int64

In [16]:

# Now, apply the np.where()
df['MaritalStatus'] = np.where(df['MaritalStatus'] == 'PARTNER', 1,
                             np.where(df['MaritalStatus'] == 'MARRIED', 1, 0))

print(df['MaritalStatus'].value_counts())


MaritalStatus
0    5876
1    4560
Name: count, dtype: int64


In [17]:
# Convert the column to strings first to handle the 'P' value
df['CivilStatus'] = df['CivilStatus'].astype(str)

# Convert all non-numeric values to a placeholder, e.g., '-999'
df['CivilStatus'] = pd.to_numeric(df['CivilStatus'], errors='coerce').fillna(-999).astype(int)

# Now, apply the np.where()
df['CivilStatus'] = np.where(df['CivilStatus'] == 2, 2,
                             np.where(df['CivilStatus'] == 4, 1, 0))

print(df['CivilStatus'].value_counts())


CivilStatus
0    7412
1    1619
2    1405
Name: count, dtype: int64


## Add Vilja

In [18]:
Vilja['degenerate_application_id'] =Vilja['degenerate_application_id'].astype(str)

# Left join
df = df.merge(Vilja, 
                  left_on='ApplicationID', 
                  right_on='degenerate_application_id', 
                  how='left')

## Filter away accounts before Vilja was implemented

In [19]:
# Ensure the 'DisbursedDate' column is of datetime64 type
df['DisbursedDate'] = pd.to_datetime(df['DisbursedDate'])

# Filter the DataFrame
df = df[df['DisbursedDate'] > '2017-02-24']






## SORT IT BY DATE!

In [20]:
df['ReceivedDate'] = pd.to_datetime(df['ReceivedDate'])
df = df.sort_values(by='ReceivedDate')


In [21]:
for now in range(len(df['ReceivedDate'])-1):

    if df['ReceivedDate'].iloc[now] > df['ReceivedDate'].iloc[now+1]:
        print('NOT Sorted')






## save for later 
d = df['ReceivedDate'] 

d.to_csv('ReceivedDate.csv', index=False)

#df = df.drop(columns='ReceivedDate')


## Filter away accounts that are not working with our normal join logic, for many of these it we dont have a cretit report. Karin says that they might have taking a report manually and then we dont see it in the data  

In [22]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
missing_cbr = df[df['TotalIncome'].isna()]

## få fram SSN 
#missing_cbr.SSN_A.iloc[2]

## innan vi hade vilja

print(missing_cbr.Ever90.value_counts())

missing_cbr.head()


Ever90
0    2
Name: count, dtype: int64


,Ever90,AccountNumber,ApplicationID,SSN_A,IsMainApplicant,HasCoapp,ReceivedDate,DisbursedDate,Amount,StartupFee,ApplicationScore,PaymentRemarks,CreditOfficer,SalesChannel,Product,Migrated,BrokerName,OriginalSalesChannel,BirthDate,Bookingtype,MaritalStatus,EmploymentType,HousingType,MonthlySalary,Referer,Campaign,SourceMedium,Keyword,NystartChannel,PNReceivedDate,NumberOfApplicants,Gender,CoappSameAddress,Kronofogden,CreditCardsNo,InstallmentLoansNo,UnsecuredLoansNo,LastPaymentRemarkDate1,TotalLoans,NystartBalance,TotalUnsecuredLoans,UCScore,SSN,jsonID,Date,import_key,SSN2,Inquiries12M,CountyCode,MunicipalityCode,PostalCode,GuardianAppointed,BlockCode,BlockCodeDate,CivilStatus,CivilStatusDate,TimeOnAddress,AddressType,Country,IncomeYear,ActiveBusinessIncome,PassiveBusinessIncome,EmploymentIncome,CapitalIncome,CapitalDeficit,GeneralDeductions,ActiveBusinessDeficit,TotalIncome,IncomeYear2,ActiveBusinessIncome2,PassiveBusinessIncome2,EmploymentIncome2,CapitalIncome2,CapitalDeficit2,GeneralDeductions2,ActiveBusinessDeficit2,TotalIncome2,IncomeBeforeTax,IncomeBeforeTaxPrev,IncomeFromCapital,DeficitFromCapital,IncomeFromOwnBusiness,PaymentRemarksNo,PaymentRemarksAmount,LastPaymentRemarkDate,KFMPublicClaimsAmount,KFMPrivateClaimsAmount,KFMTotalAmount,KFMPublicClaimsNo,KFMPrivateClaimsNo,HouseTaxValue,HouseOwnershipPct,HouseOwnershipStatus,HouseOwnershipNo,BusinessInquiries,CreditCardsUtilizationRatio,HasMortgageLoan,HasCard,HasUnsecuredLoan,HasInstallmentLoan,IndebtednessRatio,AvgIndebtednessRatio12M,ActiveCreditAccounts,NewUnsecuredLoans12M,NewInstallmentLoans12M,NewCreditAccounts12M,NewMortgageLoans12M,TotalNewExMortgage12M,VolumeChange12MExMortgage,VolumeChange12MUnsecuredLoans,VolumeChange12MInstallmentLoans,VolumeChange12MCreditAccounts,VolumeChange12MMortgageLoans,AvgUtilizationRatio12M,VolumeUsed,NumberOfAccounts,NumberOfLenders,ApprovedCreditVolume,InstallmentLoansVolume,CreditAccountsVolume,UnsecuredLoansVolume,MortgageLoansHouseVolume,MortgageLoansApartmentVolume,NumberOfCredits,NumberOfCreditors,ApprovedCardsLimit,NumberOfCreditCards,NumberOfBlancoLoans,SharedVolumeExMortgage,SharedVolume,NumberOfUnsecuredLoans,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards,Applicationtype,degenerate_application_id,kids_number,housing_cost
7220,0,5040415,5040415,20FF3F74700A01F4A440F1C728F1F09812A48809931B3B...,0,1,2017-10-16,2017-11-20,65000.0,NaN,NaN,0,Malcolm,WEB,Nystartlånet,0,None,WEB,1988-11-29,New Loan,0,FULL,RENTED,32000.0,None,Missing,Missing,Missing,Undefined,2017-10-31,2,Female,0,0.0,NaN,NaN,NaN,None,0.0,NaN,0.0,0.044,NaN,59edcf2bf04d330001c37805,2017-10-23,59edcf2bf04d330001c3780520171023,8811290603,2,01,80,11423,N,None,None,0,None,2016-11-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,0.0,None,None,None,None,None,None,None,None,None,None,None,None,0,None,None,0.0,0.0,0.0,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,0,0,None,0,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,NaN,None,None,None,None,2.0,5040415,2.0,3500.0
7339,0,5186630,5186630,4DF763CB2C759C33B20A72EE1E8D3CB34A8564D3A2BE62...,0,1,2018-01-25,2018-02-28,145000.0,NaN,NaN,0,Mark,BROKER,Nystartlånet,0,Zensum,BROKER,1994-04-05,New Loan,1,FULL,RENTED,13500.0,None,Missing,Missing,Missing,Undefined,2018-02-28,2,Female,1,0.0,NaN,NaN,NaN,None,0.0,NaN,0.0,0.072,NaN,5a699e15ea40310001be713d,2018-01-25,5a699e15ea40310001be713d20180125,9404059660,4,12,33,23594,N,None,None,0,None,2017-11-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,0.0,None,None,None,None,None,None,None,None,None,None,None,None,0,None,None,0.0,0.0,0.0,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,0,0,No

In [23]:
df =  df[~pd.isna(df.TotalIncome)]

## Create Age Feature

In [24]:

# Get today's date without time
today = pd.Timestamp('today').floor('D')

df['BirthDate'] = pd.to_datetime(df['BirthDate'])

# Compute the age based solely on years
df['age'] = today.year -  df['BirthDate'].dt.year

# Adjust for cases where the birthdate hasn't occurred this year yet
df['age'] = np.where((today.month < df['BirthDate'].dt.month) | 
                     ((today.month == df['BirthDate'].dt.month) & (today.day < df['BirthDate'].dt.day)), 
                     df['age']-1, 
                     df['age'])



Data Analys Packadge on Features

## Clean and prepare data

# Create Final dataset

### Feature Engineering - LastPaymentRemarkDate_New

In [25]:
# df_without_nans = df.dropna(subset=['LastPaymentRemarkDate_New']).copy()
# df_with_nans = df[df['LastPaymentRemarkDate_New'].isna()].copy()

# df_without_nans['LastPaymentRemarkDate_New'] = pd.to_datetime(df_without_nans['LastPaymentRemarkDate_New'])

# LastPaymentRemarkDate_DistanceDays = df_without_nans.ReceivedDate - df_without_nans.LastPaymentRemarkDate_New
# LastPaymentRemarkDate_DistanceDays = pd.to_timedelta(LastPaymentRemarkDate_DistanceDays).dt.days

# df_without_nans.loc[:, 'LastPaymentRemarkDate_DistanceDays'] = LastPaymentRemarkDate_DistanceDays
# df_with_nans.loc[:, 'LastPaymentRemarkDate_DistanceDays'] = max(LastPaymentRemarkDate_DistanceDays) * 1.4

# df = pd.concat([df_without_nans, df_with_nans])


In [26]:


droplist = [ # 'AccountNumber'                                                         ## SAme as Application ID
            
            #,'ReceivedDate'


            'SSN_A',
            'IsMainApplicant','HasCoapp','NumberOfApplicants','CoappSameAddress' #  skapat en dummy för dessa
            ,'BirthDate'
            ,'Campaign'                                                             ## TO many different chategories
            ,'DisbursedDate'                                                       # och ha recived för det var då vi fick data
            ,'CreditOfficer'
            ,'Product','OriginalSalesChannel','bookingtype'                        #   TA bort Produkt och ha Sales Channel (allt är sanna produkt...)--> RICHARD SA VAR BÄTTRE även OriginalSalesChannel , dennna är samma ish som sales channel även bookingtype
            ,'Migrated'
            ,'Referer'                                                             # no data
            ,'Bookingtype' 
            #,'MaritalStatus'                                                      # take UC variable CivilStatus instead
            ,'EmploymentType'
            ,'IncomeYear'
            ,'IncomeYear2'
            ,'ActiveBusinessIncome'
            ,'ActiveBusinessIncome2'                                               ## för få obs
            ,'PassiveBusinessIncome'                                              ## för få obs                               
            ,'GeneralDeductions'
            ,'GeneralDeductions2'  
                                                                                                                     
            ,'SourceMedium' 
            ,'Keyword'  
             ,'NystartChannel' 
            ,'PNReceivedDate'
            ,'NystartBalance'                                                   ## för lite data                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    ## data excist but so few in each class so bad variable
            ,'SSN'
            ,'jsonID'
            ,'Date'
            ,'import_key'
            ,'SSN2'
            ,'CountyCode'
            #,'LastPaymentRemarkDate'
            ,'LastPaymentRemarkDate_New'
            ,'PostalCode'
            ,'BlockCode'                                                        ## almost no data
            ,'BlockCodeDate'
            ,'AddressType'                                                         ## almost no data
            ,'Country'   
            ,'RiskPrognos'                                                      ## UCscore has data everywhere, not the case for CBR data
            ,'IncomeYear'
            #,'TimeOnAddress'
            ,'ActiveBusinessDeficit'                                            ## almost no data
            #,'MortgageLoansApartmentVolume'
            ,'ActiveBusinessDeficit'
            ,'CivilStatusDate'                                                  ## distrubution of data is weird



            ## ALL THESE ARE dupplicated featrures that the same df already have...
            ,'IncomeBeforeTax'
            ,'IncomeBeforeTaxPrev'
            ,'IncomeFromCapital'
            ,'DeficitFromCapital'
            ,'IncomeFromOwnBusiness'
            
            
            ,'HouseOwnershipStatus'                                                ## almost no data
            ,'BusinessInquiries'                                                    ## almost no data
            'HasMortgageLoan'                                                       ## no data
            
            


            ,'GuardianAppointed'                                                    ## <jusy one class so no data....
            
            ,'ApplicationID'
            
            ,'MunicipalityCode'

            ,'LastPaymentRemarkDate_New'
            ,'LastPaymentRemarkDate1'
,'HasMortgageLoan'
            ]




### dubbelkolla och hämta senare

## TimeOnAddress  
# 
# MortgageLoansApartmentVolume, ActiveBusinessDeficit ,  ActiveBusinessDeficit2     THIS IS UC DATA , exaclty 1.5 % that corresponds to the difference that is missing from score, income etc, looks like an inscorrect join where data gets lost...

df = drop_columns(df, droplist)


MonthlySalary vad de säger?,                EmploymentIncome	CapitalIncome , TotalIncome	PassiveBusinessIncome2	EmploymentIncome2	
                    ,PassiveBusinessIncome2	EmploymentIncome2	CapitalIncome2	CapitalDeficit2	ActiveBusinessDeficit2	TotalIncome2	

In [27]:
print(len(df))
df[3000:3010]

10401


,Ever90,AccountNumber,ReceivedDate,Amount,StartupFee,ApplicationScore,PaymentRemarks,SalesChannel,BrokerName,MaritalStatus,HousingType,MonthlySalary,Gender,Kronofogden,CreditCardsNo,InstallmentLoansNo,UnsecuredLoansNo,TotalLoans,TotalUnsecuredLoans,UCScore,Inquiries12M,CivilStatus,TimeOnAddress,EmploymentIncome,CapitalIncome,CapitalDeficit,TotalIncome,PassiveBusinessIncome2,EmploymentIncome2,CapitalIncome2,CapitalDeficit2,ActiveBusinessDeficit2,TotalIncome2,PaymentRemarksNo,PaymentRemarksAmount,LastPaymentRemarkDate,KFMPublicClaimsAmount,KFMPrivateClaimsAmount,KFMTotalAmount,KFMPublicClaimsNo,KFMPrivateClaimsNo,HouseTaxValue,HouseOwnershipPct,HouseOwnershipNo,BusinessInquiries,CreditCardsUtilizationRatio,HasCard,HasUnsecuredLoan,HasInstallmentLoan,IndebtednessRatio,AvgIndebtednessRatio12M,ActiveCreditAccounts,NewUnsecuredLoans12M,NewInstallmentLoans12M,NewCreditAccounts12M,NewMortgageLoans12M,TotalNewExMortgage12M,VolumeChange12MExMortgage,VolumeChange12MUnsecuredLoans,VolumeChange12MInstallmentLoans,VolumeChange12MCreditAccounts,VolumeChange12MMortgageLoans,AvgUtilizationRatio12M,VolumeUsed,NumberOfAccounts,NumberOfLenders,ApprovedCreditVolume,InstallmentLoansVolume,CreditAccountsVolume,UnsecuredLoansVolume,MortgageLoansHouseVolume,MortgageLoansApartmentVolume,NumberOfCredits,NumberOfCreditors,ApprovedCardsLimit,NumberOfCreditCards,NumberOfBlancoLoans,SharedVolumeExMortgage,SharedVolume,NumberOfUnsecuredLoans,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards,Applicationtype,degenerate_application_id,kids_number,housing_cost,age
8397,0,7100993,2019-09-17,400000.0,NaN,NaN,1,WEB,None,0,None,33500.0,Male,1.0,1.0,NaN,1.0,1397177.0,57758.0,0.245,1,2,2018-11-01,438,None,29,409,None,431,None,23,None,408,3,158823,2019-04-24,0,8911,8911,0,1,1200,50,1,0,950,1.0,1.0,0.0,0.1198,0.2079,1,0,0,0,0,None,NaN,-0.86,0.0,-0.05,0.0,0.95,1397177,5,2,1398414,None,23763,28736,1344678,None,2,2,25000,1,1.0,38131,39368,1,28736,None,None,1,0,95.05,Nej,3,1344678,0,1.0,7556848,0.0,3800.0,47
8398,0,7556913,2019-09-17,290000.0,NaN,NaN,0,WEB,None,1,OWNED,26800.0,Female,0.0,1.0,NaN,2.0,98931.0,98931.0,0.005,3,0,2018-11-01,351,0,28,324,None,344,32,0,None,375,None,None,None,None,None,None,None,None,None,None,None,0,10,1.0,1.0,0.0,0.2816,0.1364,1,2,0,0,None,2,3.13,1.00,0.0,-0.96,NaN,0.42,98931,3,1,118720,None,211,98720,None,None,3,1,20000,1,2.0,98931,118720,2,0,None,None,1,0,1.06,Nej,None,None,0,2.0,7556913,0.0,2000.0,28
4539,0,7555303,2019-09-17,285000.0,1995.0,2.230471,1,WEB,None,0,RENTED,32000.0,Female,1.0,1.0,NaN,2.0,15620.0,15620.0,0.389,1,0,2018-11-01,383,None,59,324,None,360,None,63,None,296,1,0,2018-09-29,0,278620,278620,0,16,None,None,None,0,1073,1.0,1.0,0.0,0.0407,0.0665,1,0,0,0,None,0,-0.76,-0.49,0.0,-0.90,NaN,1.16,15620,3,2,15325,None,4295,11325,None,None,3,2,4000,1,2.0,15620,15325,2,0,None,None,1,0,107.38,Ja,None,None,0,0.0,7555303,0.0,5500.0,63
4538,0,7554124,2019-09-17,70000.0,995.0,3.217380,1,BROKER,Axo,1,OWNED,27300.0,Female,0.0,2.0,NaN,1.0,30946.0,30946.0,0.122,12,0,2018-11-01,380,None,9,371,None,370,None,5,None,365,2,24896,2017-07-11,None,None,None,None,None,None,None,None,0,967,1.0,1.0,0.0,0.0813,0.0444,2,1,0,0,None,1,3.40,1.00,0.0,0.65,NaN,0.72,30946,3,2,31335,None,11611,19335,None,None,3,2,12000,2,1.0,30946,31335,1,0,None,None,2,0,96.76,Nej,None,None,0,0.0,7554124,0.0,3087.0,37
8399,0,7557168,2019-09-17,160000.0,NaN,NaN,1,WEB,None,0,None,28000.0,Male,0.0,NaN,NaN,1.0,16192.0,16192.0,0.061,5,2,2018-11-01,358,None,4,354,None,391,None,10,None,380,2,2556,2017-06-15,None,None,None,None,None,None,None,None,0,0,0.0,1.0,0.0,0.0452,0.0411,0,1,0,0,None,1,1.00,1.00,0.0,0.00,NaN,NaN,16192,1,1,16192,None,None,16192,None,None,1,1,None,None,1.0,16192,16192,1,0,None,None,None,None,NaN,None,None,None,None,1.0,7557168,2.0,5800.0,33
4537,0,7553613,2019-09-17,100000.0,995

In [28]:
df.TimeOnAddress.value_counts()

TimeOnAddress
2020-11-01    2570
2021-11-01    2307
2019-11-01    2024
2018-11-01    1863
2017-11-01    1135
2016-11-01     502
Name: count, dtype: int64

In [29]:
import pandas as pd

# Convert 'TimeOnAddress' to a datetime format if it's not already
df['TimeOnAddress'] = pd.to_datetime(df['TimeOnAddress'])

# Define today's date
today = pd.Timestamp.now() # normalize() sets the time to midnight, removing hours, minutes, and seconds

# Calculate the difference
df['TimeOnAddress'] = (today - df['TimeOnAddress']).dt.days  # This will give the difference in days


In [30]:
df['TimeOnAddress'].value_counts()

TimeOnAddress
1094    2570
729     2307
1460    2024
1825    1863
2190    1135
2555     502
Name: count, dtype: int64

In [31]:
import pandas as pd

# Convert 'TimeOnAddress' to a datetime format if it's not already
df['LastPaymentRemarkDate'] = pd.to_datetime(df['LastPaymentRemarkDate'])

# Calculate the difference
df['LastPaymentRemarkDate'] = (today - df['LastPaymentRemarkDate']).dt.days  # This will give the difference in days


In [32]:
df['LastPaymentRemarkDate'] 

7025       NaN
7026       NaN
7028    2577.0
7032       NaN
7038    2511.0
         ...  
7013       NaN
7008    1082.0
7009       NaN
7011    1016.0
7014       NaN
Name: LastPaymentRemarkDate, Length: 10401, dtype: float64

In [33]:
## Create IncomeDelta Feature

## Filter to do calculations and merge after
df_without_nans = df.dropna(subset=['TotalIncome'])

df_with_nans = df[df['TotalIncome'].isna()]


df_without_nans['TotalIncome'] = df_without_nans['TotalIncome'].astype(float)
df_without_nans['TotalIncome2'] = df_without_nans['TotalIncome2'].astype(float)

df_without_nans['IncomeDelta_1Year'] = (df_without_nans.TotalIncome - df_without_nans.TotalIncome2) /df_without_nans.TotalIncome2 

df = pd.concat([df_without_nans,df_with_nans])


df.IncomeDelta_1Year

/var/folders/xv/px633rpn27q5n8m3579ly8dm0000gn/T/ipykernel_4902/3447537336.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_without_nans,df_with_nans])


7025   -0.250000
7026    1.272727
7028    0.054124
7032   -0.042763
7038    0.214022
          ...   
7013    0.496599
7008    0.227273
7009   -0.048889
7011    0.017778
7014    0.369048
Name: IncomeDelta_1Year, Length: 10401, dtype: float64

In [34]:
## Create IncomeDelta Feature

## Filter to do calculations and merge after
df_without_nans = df.dropna(subset=['TotalIncome'])

df_with_nans = df[df['TotalIncome'].isna()]


df_without_nans['TotalIncome'] = df_without_nans['TotalIncome'].astype(float)
df_without_nans['TotalIncome2'] = df_without_nans['TotalIncome2'].astype(float)

df_without_nans['IncomeDelta_1Year'] = (df_without_nans.TotalIncome - df_without_nans.TotalIncome2) /df_without_nans.TotalIncome2 

df = pd.concat([df_without_nans,df_with_nans])


df.IncomeDelta_1Year

7025   -0.250000
7026    1.272727
7028    0.054124
7032   -0.042763
7038    0.214022
          ...   
7013    0.496599
7008    0.227273
7009   -0.048889
7011    0.017778
7014    0.369048
Name: IncomeDelta_1Year, Length: 10401, dtype: float64

## Feature Engineering CapitalDeficit_Delta_1Year

In [35]:
df['CapitalDeficit'] = pd.to_numeric(df['CapitalDeficit'], errors='coerce')
df['CapitalDeficit2'] = pd.to_numeric(df['CapitalDeficit2'], errors='coerce')

# Create a mask for rows where both columns are not NaN
mask = df['CapitalDeficit'].notna() & df['CapitalDeficit2'].notna()

# Use the mask to target these rows and do the calculation
df.loc[mask, 'CapitalDeficit_Delta_1Year'] = (df.loc[mask, 'CapitalDeficit'] - df.loc[mask, 'CapitalDeficit2']) / df.loc[mask, 'CapitalDeficit2']

df['CapitalDeficit_Delta_1Year']


7025         NaN
7026         NaN
7028         NaN
7032    0.260870
7038   -0.333333
          ...   
7013    1.000000
7008         inf
7009    0.202381
7011    0.596491
7014    0.500000
Name: CapitalDeficit_Delta_1Year, Length: 10401, dtype: float64

In [36]:
df['KFMPrivateClaimsAmount']

7025    None
7026    None
7028    None
7032    None
7038    None
        ... 
7013    None
7008    None
7009    None
7011    None
7014    None
Name: KFMPrivateClaimsAmount, Length: 10401, dtype: object

## Fill nan´s etc

In [37]:
credit_data_columns = [
    'PaymentRemarksNo',
    'PaymentRemarksAmount',
    "CreditCardsNo",
    "ApprovedCardsLimit",
    "CreditAccountsVolume",
    "CapitalIncome",
    "PassiveBusinessIncome2",
    "CapitalIncome2",
    "ActiveBusinessDeficit2",
    "KFMPublicClaimsAmount",
    "KFMTotalAmount",
    'KFMPrivateClaimsAmount',   # Added the missing comma here
    "KFMPublicClaimsNo",
    "KFMPrivateClaimsNo",
    "HouseTaxValue",
    "MortgageLoansHouseVolume",
    'AvgUtilizationRatio12M',
    'EmploymentIncome',
    'EmploymentIncome2'

    ,'TimeOnAddress'
    ,'LastPaymentRemarkDate'

    ,'Ever90'
]



# Ensure the specified columns are float and fill NaN with 0
for column in credit_data_columns:
    if column in df.columns:  # Only apply to columns that exist in the dataframe
        df[column] = df[column].astype(float).fillna(0)


In [38]:
df['KFMPrivateClaimsAmount']

7025    0.0
7026    0.0
7028    0.0
7032    0.0
7038    0.0
       ... 
7013    0.0
7008    0.0
7009    0.0
7011    0.0
7014    0.0
Name: KFMPrivateClaimsAmount, Length: 10401, dtype: float64

In [39]:
loan_columns = [
    "InstallmentLoansNo",
    "IndebtednessRatio",
    "AvgIndebtednessRatio12M",
    "InstallmentLoansVolume",
    "VolumeChange12MExMortgage",
    "VolumeChange12MUnsecuredLoans",
    "VolumeChange12MInstallmentLoans",
    "VolumeChange12MCreditAccounts",
    "VolumeChange12MMortgageLoans",
    "AvgUtilizationRatio12M",
    "CreditCardsUtilizationRatio",
    "UnsecuredLoansVolume",
    "NumberOfLenders",
    "CapitalDeficit",
    "CapitalDeficit2",
    "NewUnsecuredLoans12M",
    "NewInstallmentLoans12M",
    "NewCreditAccounts12M",
    "VolumeUsed",
    "ApprovedCreditVolume"
    ,'NumberOfBlancoLoans'
    ,'NumberOfCreditCards'
    ,'NewMortgageLoans12M'
    ,	'TotalNewExMortgage12M'

     ,  "NumberOfMortgageLoans",
    "SharedVolumeMortgageLoans",
    "SharedVolumeCreditCards",
    "NumberOfUnsecuredLoans",
    "SharedVolumeUnsecuredLoans",
    "NumberOfInstallmentLoans",
    "SharedVolumeInstallmentLoans",
    "NumberOfCreditAccounts",
    "SharedVolumeCrerditAccounts"
    ,'UnsecuredLoansNo'
    , 'IncomeDelta_1Year'
    ,'kids_number'


,'AccountNumber'
, 'Inquiries12M'
    ,'BusinessInquiries'
    , 'ActiveCreditAccounts'
    , 'NumberOfAccounts'
    ,'NumberOfCredits'
    , 'NumberOfCreditors'
    , 'SharedVolumeExMortgage'
    ,'SharedVolume'
    , 'degenerate_application_id'

    ,'ApplicationScore'

    ,'UCScore'
    ,'StartupFee'

    


  ]



# Ensure the specified columns are float and fill NaN with -1
for column in loan_columns:
    if column in df.columns:  # Only apply to columns that exist in the dataframe
        df[column] = df[column].astype(float).fillna(-1)


## These may disturb the distribution so -100 is more of

In [40]:
loan_columns = [
"CapitalDeficit_Delta_1Year","UtilizationRatio",'housing_cost']



# Ensure the specified columns are float and fill NaN with -1
for column in loan_columns:
    if column in df.columns:  # Only apply to columns that exist in the dataframe
        df[column] = df[column].astype(float).fillna(-100)



inf_columns = ['CapitalDeficit_Delta_1Year',
               'IncomeDelta_1Year',
               'ActiveCreditAccounts']
      
for col in inf_columns:
    if col in df.columns:
        df[col] = df[col].replace([np.inf, -np.inf], -100)


In [41]:
df


,Ever90,AccountNumber,ReceivedDate,Amount,StartupFee,ApplicationScore,PaymentRemarks,SalesChannel,BrokerName,MaritalStatus,HousingType,MonthlySalary,Gender,Kronofogden,CreditCardsNo,InstallmentLoansNo,UnsecuredLoansNo,TotalLoans,TotalUnsecuredLoans,UCScore,Inquiries12M,CivilStatus,TimeOnAddress,EmploymentIncome,CapitalIncome,CapitalDeficit,TotalIncome,PassiveBusinessIncome2,EmploymentIncome2,CapitalIncome2,CapitalDeficit2,ActiveBusinessDeficit2,TotalIncome2,PaymentRemarksNo,PaymentRemarksAmount,LastPaymentRemarkDate,KFMPublicClaimsAmount,KFMPrivateClaimsAmount,KFMTotalAmount,KFMPublicClaimsNo,KFMPrivateClaimsNo,HouseTaxValue,HouseOwnershipPct,HouseOwnershipNo,BusinessInquiries,CreditCardsUtilizationRatio,HasCard,HasUnsecuredLoan,HasInstallmentLoan,IndebtednessRatio,AvgIndebtednessRatio12M,ActiveCreditAccounts,NewUnsecuredLoans12M,NewInstallmentLoans12M,NewCreditAccounts12M,NewMortgageLoans12M,TotalNewExMortgage12M,VolumeChange12MExMortgage,VolumeChange12MUnsecuredLoans,VolumeChange12MInstallmentLoans,VolumeChange12MCreditAccounts,VolumeChange12MMortgageLoans,AvgUtilizationRatio12M,VolumeUsed,NumberOfAccounts,NumberOfLenders,ApprovedCreditVolume,InstallmentLoansVolume,CreditAccountsVolume,UnsecuredLoansVolume,MortgageLoansHouseVolume,MortgageLoansApartmentVolume,NumberOfCredits,NumberOfCreditors,ApprovedCardsLimit,NumberOfCreditCards,NumberOfBlancoLoans,SharedVolumeExMortgage,SharedVolume,NumberOfUnsecuredLoans,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards,Applicationtype,degenerate_application_id,kids_number,housing_cost,age,IncomeDelta_1Year,CapitalDeficit_Delta_1Year
7025,0.0,5005418.0,2017-02-02,350000.0,-1.0,-1.000000,0,BROKER,Zensum,1,HOUSE,15000.0,Female,0.0,0.0,-1.0,-1.0,0.0,0.0,0.042,4.0,0,2555.0,33.0,0.0,-1.0,33.0,0.0,44.0,0.0,-1.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,0.0,-1.0,0.0,0.0,0.0,-1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,0.00,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-100.00,None,-1.0,-1.0,-1.0,1.0,5005418.0,3.0,1000.0,44,-0.250000,-100.000000
7026,0.0,5005574.0,2017-02-06,125000.0,-1.0,-1.000000,0,BROKER,Zensum,0,RENTED,22000.0,Male,0.0,0.0,-1.0,-1.0,0.0,0.0,0.018,1.0,0,2555.0,100.0,0.0,-1.0,100.0,0.0,44.0,0.0,-1.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,0.0,-1.0,0.0,0.0,0.0,-1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,0.00,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-100.00,None,-1.0,-1.0,-1.0,2.0,5005574.0,1.0,5000.0,32,1.272727,-100.000000
7028,1.0,5005806.0,2017-02-07,400000.0,-1.0,-1.000000,1,BROKER,Zensum,1,RENTED,35000.0,Male,1.0,0.0,-1.0,-1.0,0.0,0.0,0.443,1.0,1,2555.0,410.0,0.0,0.0,409.0,0.0,389.0,0.0,0.0,0.0,388.0,9.0,6990.0,2577.0,0.0,0.0,0.0,0.0,0.0,112.0,100,1,0.0,-1.0,0.0,0.0,0.0,-1.0000,-1.0000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.00,-1.0,0.00,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,None,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-100.00,None,-1.0,-1.0,-1.0,1.0,5005806.0,0.0,3000.0,60,0.054124,-100.000000
7032,0.0,5006028.0,2017-02-08,205000.0,-1.0,-1.000000,0,BROKER,Zensum,1,RENTED,31000.0,Male,0.0,2.0,-1.0,4.0,338276.0,338276.0,0.035,7.0,2,2555.0,320.0,0.0,29.0,291.0,0.0,326.0,0.0,23.0,0.0,304.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,0.0,403.0,1.0,1.0,0.0,1.0571,1.0137,2.0,1.0,0.0,0.0,-1.0,1.0,0.15,0.21,0.00,-0.69,-1.0,0.83,338276.0,6.0,4.0,347994.0,-1.0,6582.0,331694.0,0.0,None,6.0,4.0,16300.0,2.0,4.0,338276.0,347994.0,4.0,0.0,-1.0,-1.0,2.0,0.0,40.38,Nej,-1.0,-1.0,0.0,1.0,5006028.0,0.0,2250.0,53,-0.042763,0.260870
7038,0.0,5006556.0,2017-02-14,375000.0,-1.0,-1.000000,1,BROKER,Zensum,1,HOUSE,32700.0,Male,1.0,1.0,1.0,5.0,590158.0,590158.0,0.527,24.

In [42]:
## the rest

for Cname in df.columns:

    if str(df[Cname].dtype) == 'object':
        df[Cname].fillna(-1, inplace=True)
        df[Cname].replace('None', -1, inplace=True)

In [43]:
# import pandas_profiling

# profile = pandas_profiling.ProfileReport(df, minimal=True)  # Use minimal mode for large datasets
# profile.to_file("main_report_28_aug.html")

which one first?

0. One hot encoding, 
1. Min-Max scaling

2. Some Feature selection methods



3. df = pd.get_dummies(df, columns=['channel'], drop_first=True)


In [44]:
df.CivilStatus.value_counts()

CivilStatus
0    7385
1    1614
2    1402
Name: count, dtype: int64

## One Hot encoding - Only Chategorical Features the are not ordinal - Dont FORRGET THAT THIS MIGHT NOT BE THE REIGHT WAY TO GO LATER WHEN DOING DUMMIES, then I should one hot encode after WoE

In [45]:
from sklearn.preprocessing import LabelEncoder

# Specify the columns to one-hot encode
columns_to_encode = [
    'BrokerName',
    "SalesChannel", 
    "HousingType", 
    "Gender", 
    "HouseOwnershipPct", 
    "HouseOwnershipNo", 
    "CreditAccountOverdraft",
    'MaritalStatus'
]


for column in columns_to_encode:
    df[column] = df[column].astype(str)  # Convert to string
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])


In [46]:
df.CivilStatus.value_counts()

CivilStatus
0    7385
1    1614
2    1402
Name: count, dtype: int64

In [47]:
df.MaritalStatus.value_counts()

print(df[df.AccountNumber ==  7798325].Inquiries12M)
df[df.AccountNumber ==  7798325]

6831    5.0
Name: Inquiries12M, dtype: float64


,Ever90,AccountNumber,ReceivedDate,Amount,StartupFee,ApplicationScore,PaymentRemarks,SalesChannel,BrokerName,MaritalStatus,HousingType,MonthlySalary,Gender,Kronofogden,CreditCardsNo,InstallmentLoansNo,UnsecuredLoansNo,TotalLoans,TotalUnsecuredLoans,UCScore,Inquiries12M,CivilStatus,TimeOnAddress,EmploymentIncome,CapitalIncome,CapitalDeficit,TotalIncome,PassiveBusinessIncome2,EmploymentIncome2,CapitalIncome2,CapitalDeficit2,ActiveBusinessDeficit2,TotalIncome2,PaymentRemarksNo,PaymentRemarksAmount,LastPaymentRemarkDate,KFMPublicClaimsAmount,KFMPrivateClaimsAmount,KFMTotalAmount,KFMPublicClaimsNo,KFMPrivateClaimsNo,HouseTaxValue,HouseOwnershipPct,HouseOwnershipNo,BusinessInquiries,CreditCardsUtilizationRatio,HasCard,HasUnsecuredLoan,HasInstallmentLoan,IndebtednessRatio,AvgIndebtednessRatio12M,ActiveCreditAccounts,NewUnsecuredLoans12M,NewInstallmentLoans12M,NewCreditAccounts12M,NewMortgageLoans12M,TotalNewExMortgage12M,VolumeChange12MExMortgage,VolumeChange12MUnsecuredLoans,VolumeChange12MInstallmentLoans,VolumeChange12MCreditAccounts,VolumeChange12MMortgageLoans,AvgUtilizationRatio12M,VolumeUsed,NumberOfAccounts,NumberOfLenders,ApprovedCreditVolume,InstallmentLoansVolume,CreditAccountsVolume,UnsecuredLoansVolume,MortgageLoansHouseVolume,MortgageLoansApartmentVolume,NumberOfCredits,NumberOfCreditors,ApprovedCardsLimit,NumberOfCreditCards,NumberOfBlancoLoans,SharedVolumeExMortgage,SharedVolume,NumberOfUnsecuredLoans,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards,Applicationtype,degenerate_application_id,kids_number,housing_cost,age,IncomeDelta_1Year,CapitalDeficit_Delta_1Year
6831,0.0,7798325.0,2022-08-25,185000.0,995.0,2.517761,1,0,8,0,5,28300.0,0,0.0,1.0,1.0,1.0,147385.0,147385.0,0.253,5.0,0,729.0,340.0,0.0,1.0,339.0,0.0,389.0,0.0,0.0,0.0,388.0,6.0,3360.0,1096.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,896.0,1.0,1.0,1.0,0.4336,0.1995,1.0,1.0,1.0,1.0,0.0,-1.0,1.0,1.0,1.0,1.0,0.0,0.56,147385.0,3.0,2.0,152549.0,8815.0,44836.0,93734.0,0.0,-1,3.0,2.0,50000.0,1.0,1.0,147385.0,152549.0,1.0,0.0,1.0,0.0,1.0,0.0,89.67,2,-1.0,-1.0,0.0,0.0,8359232.0,1.0,6676.0,44,-0.126289,-100.0


## Below only messed up stuff

In [48]:
# # non_numeric_columns = df.select_dtypes(include=['object']).columns
# # print(non_numeric_columns)

# # Identify columns that are of type 'object' or 'category'
# categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# categorical_columns


# # Try to convert these columns to float
# for col in categorical_columns:
#     try:
#         df[col] = df[col].astype(float)
#     except ValueError:
#         # For columns that can't be converted directly, we might need a mapping
#         # Here's a simple example mapping, but this will vary based on your data
#         mapping = {value: float(index) for index, value in enumerate(df[col].unique())}
#         df[col] = df[col].map(mapping)

In [49]:
print(df[df.AccountNumber ==  7798325].Inquiries12M)
df[df.AccountNumber ==  7798325] #.Inquiries12M




6831    5.0
Name: Inquiries12M, dtype: float64


,Ever90,AccountNumber,ReceivedDate,Amount,StartupFee,ApplicationScore,PaymentRemarks,SalesChannel,BrokerName,MaritalStatus,HousingType,MonthlySalary,Gender,Kronofogden,CreditCardsNo,InstallmentLoansNo,UnsecuredLoansNo,TotalLoans,TotalUnsecuredLoans,UCScore,Inquiries12M,CivilStatus,TimeOnAddress,EmploymentIncome,CapitalIncome,CapitalDeficit,TotalIncome,PassiveBusinessIncome2,EmploymentIncome2,CapitalIncome2,CapitalDeficit2,ActiveBusinessDeficit2,TotalIncome2,PaymentRemarksNo,PaymentRemarksAmount,LastPaymentRemarkDate,KFMPublicClaimsAmount,KFMPrivateClaimsAmount,KFMTotalAmount,KFMPublicClaimsNo,KFMPrivateClaimsNo,HouseTaxValue,HouseOwnershipPct,HouseOwnershipNo,BusinessInquiries,CreditCardsUtilizationRatio,HasCard,HasUnsecuredLoan,HasInstallmentLoan,IndebtednessRatio,AvgIndebtednessRatio12M,ActiveCreditAccounts,NewUnsecuredLoans12M,NewInstallmentLoans12M,NewCreditAccounts12M,NewMortgageLoans12M,TotalNewExMortgage12M,VolumeChange12MExMortgage,VolumeChange12MUnsecuredLoans,VolumeChange12MInstallmentLoans,VolumeChange12MCreditAccounts,VolumeChange12MMortgageLoans,AvgUtilizationRatio12M,VolumeUsed,NumberOfAccounts,NumberOfLenders,ApprovedCreditVolume,InstallmentLoansVolume,CreditAccountsVolume,UnsecuredLoansVolume,MortgageLoansHouseVolume,MortgageLoansApartmentVolume,NumberOfCredits,NumberOfCreditors,ApprovedCardsLimit,NumberOfCreditCards,NumberOfBlancoLoans,SharedVolumeExMortgage,SharedVolume,NumberOfUnsecuredLoans,SharedVolumeUnsecuredLoans,NumberOfInstallmentLoans,SharedVolumeInstallmentLoans,NumberOfCreditAccounts,SharedVolumeCrerditAccounts,UtilizationRatio,CreditAccountOverdraft,NumberOfMortgageLoans,SharedVolumeMortgageLoans,SharedVolumeCreditCards,Applicationtype,degenerate_application_id,kids_number,housing_cost,age,IncomeDelta_1Year,CapitalDeficit_Delta_1Year
6831,0.0,7798325.0,2022-08-25,185000.0,995.0,2.517761,1,0,8,0,5,28300.0,0,0.0,1.0,1.0,1.0,147385.0,147385.0,0.253,5.0,0,729.0,340.0,0.0,1.0,339.0,0.0,389.0,0.0,0.0,0.0,388.0,6.0,3360.0,1096.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,896.0,1.0,1.0,1.0,0.4336,0.1995,1.0,1.0,1.0,1.0,0.0,-1.0,1.0,1.0,1.0,1.0,0.0,0.56,147385.0,3.0,2.0,152549.0,8815.0,44836.0,93734.0,0.0,-1,3.0,2.0,50000.0,1.0,1.0,147385.0,152549.0,1.0,0.0,1.0,0.0,1.0,0.0,89.67,2,-1.0,-1.0,0.0,0.0,8359232.0,1.0,6676.0,44,-0.126289,-100.0


In [50]:
non_numeric_columns = df.select_dtypes(include=['object']).columns
print(non_numeric_columns)

Index(['MortgageLoansApartmentVolume'], dtype='object')


In [51]:
set(df.ActiveCreditAccounts)



{-1.0,
 0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 18.0,
 19.0}

In [52]:
df.HousingType.value_counts()

HousingType
5    6470
2    1193
1     916
4     726
3     722
0     374
Name: count, dtype: int64

## Final Check

In [53]:
# Identify columns with NaN values
nan_columns = df.columns[df.isnull().any()].tolist()
nan_columns

[]

In [54]:
# Write the filtered DataFrame to CSV
df.to_csv('pree_processed.csv', index=False)

In [55]:
df[df.AccountNumber ==  7798325].Inquiries12M



6831    5.0
Name: Inquiries12M, dtype: float64

In [56]:
max(df.SharedVolumeMortgageLoans)

8395000.0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a6ba0eaf-97fa-4fd0-ab2f-858132e990c1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

df

In [57]:
df.TimeOnAddress.value_counts()

TimeOnAddress
1079.0    2570
714.0     2307
1445.0    2024
1810.0    1863
2175.0    1135
2540.0     502
Name: count, dtype: int64

In [58]:
df.age

7025    44
7026    32
7028    60
7032    53
7038    62
        ..
7013    27
7008    46
7009    53
7011    39
7014    26
Name: age, Length: 10401, dtype: int32

In [59]:
df.MortgageLoansHouseVolume

7025    0.0
7026    0.0
7028    0.0
7032    0.0
7038    0.0
       ... 
7013    0.0
7008    0.0
7009    0.0
7011    0.0
7014    0.0
Name: MortgageLoansHouseVolume, Length: 10401, dtype: float64